Step Counting Algorithm


In [17]:
# code goes here!
from heapq import heappop, heappush
from math import inf, sqrt
from visualization import visualize_graph



class graph_vertex:
  def __init__(self, name, x, y):
    self.name = name
    self.position = (x, y)

# Set up example graph
a = graph_vertex("A", 0, 2)
b = graph_vertex("B", 1, 4)
c = graph_vertex("C", 1, 1)
d = graph_vertex("D", 2, 2)
e = graph_vertex("E", 4, 1)
f = graph_vertex("F", 5, 3)
g = graph_vertex("G", 6, 1)

euclidean_graph = {
  a: [(b, 2.2), (c, 1.4)],
  b: [(a, 2.2), (d, 2.2)],
  c: [(a, 1.4), (d, 1.4)],
  d: [(b, 2.2), (c, 1.4), (e, 2.2), (f, 3.1)],
  e: [(d, 2.2), (g, 2), (f, 2.2)],
  f: [(d, 3.1), (g, 2.2), (e, 2.2)],
  g: [(e, 2), (f, 2.2)]
}

def heuristic(start, target):
  """
    Heuristic function for A* algorithm, calculates the Euclidean distance between two vertices.

    Parameters:
        start (graph_vertex): The starting vertex.
        target (graph_vertex): The target vertex.

    Returns:
        float: The Euclidean distance between the start and target vertices.
  """
  x_distance = abs(start.position[0] - target.position[0])
  y_distance = abs(start.position[1] - target.position[1])
  return sqrt(x_distance * x_distance + y_distance * y_distance)

def stepCounting(graph, start, target_node, target_steps):
    
    #store: steps_taken, path_taken
    #at each node: generate a list of all possible options with their scores where score = steps taken + steps to next node + heuristic at next node

    steps_taken = 0
    path_taken = [start]
    current_node = start
    previous_node = start

    while current_node != target_node:
      next_nodes = graph[current_node]
      best_node = None
      best_node_score = inf

      for node in next_nodes:
        if node[0] != previous_node: #assume this is for a graph where there are no nodes with only one node adjacent - degree >= 2 for all
          node_heuristic = heuristic(node[0], target_node)
          score = steps_taken + node[1] + node_heuristic
          if abs(score - target_steps) < abs(best_node_score - target_steps):
              best_node = node
              best_node_score = score
      
      steps_taken += best_node[1]
      previous_node = current_node
      current_node = best_node[0]
      path_taken.append(current_node)

    return path_taken, steps_taken


path, steps = stepCounting(euclidean_graph, a, g, 34)
print()
for step in path:
  print(step.name)
print(steps)



A
B
D
C
A
B
D
C
A
B
D
C
A
B
D
C
A
B
D
E
G
37.4
